# POS & Syntax assignment part 1

---
## Prerequisites

In [28]:
grammar = """S -> NP VP
S -> VP
NP -> DT NN
NP -> IN NNP
NP -> DT JJ NN
NP -> PRP
VP -> VBP NP
VP -> VBP VP
VP -> VBG NP
VP -> TO VP
VP -> VB
VP -> VB NP
VP -> VBD
VP -> VBD NP
VBP -> 'am'
NNP -> 'paul'
PRP -> 'i'
IN -> 'with'
VBG -> 'talking'"""

In [29]:
sent = ['i', 'am', 'talking', 'with', 'Paul']

In [48]:
grammar = """S -> NP VP
S -> VP
NP -> DT NN
NP -> DT JJ NN
NP -> PRP
VP -> VBP VP
VP -> VBP NP
VP -> VBG NP
VP -> TO VP
VP -> VB
VP -> VB NP
NN -> 'show'
NN -> 'book'
PRP -> 'i'
VBP -> 'am'
VBG -> 'watching'
VB -> 'show'
DT -> 'a'
DT -> 'the'
MD -> 'will'"""

In [49]:
sent = ['i', 'am', 'watching', 'a', 'show']

---
## Step 1: make table

In [50]:
import re

productions = grammar.split('\n')
pos_table = {}

for production in productions:
    nonterminal, sequence = production.split(' -> ')
    
    if nonterminal not in pos_table.keys():
        pos_table[nonterminal] = []

    symbol = re.sub('\'', '', sequence.split()[0])
    if symbol not in pos_table[nonterminal]:
        pos_table[nonterminal].append(symbol)

pos_table

{'S': ['NP', 'VP'],
 'NP': ['DT', 'PRP'],
 'VP': ['VBP', 'VBG', 'TO', 'VB'],
 'NN': ['show', 'book'],
 'PRP': ['i'],
 'VBP': ['am'],
 'VBG': ['watching'],
 'VB': ['show'],
 'DT': ['a', 'the'],
 'MD': ['will']}

In [51]:
from tabulate import tabulate

print_table = [[symbol, ', '.join(pos_table[symbol])] for symbol in pos_table]

print(tabulate(print_table, headers=['Symbol', 'Left corner'], tablefmt='fancy_grid'))

╒══════════╤══════════════════╕
│ Symbol   │ Left corner      │
╞══════════╪══════════════════╡
│ S        │ NP, VP           │
├──────────┼──────────────────┤
│ NP       │ DT, PRP          │
├──────────┼──────────────────┤
│ VP       │ VBP, VBG, TO, VB │
├──────────┼──────────────────┤
│ NN       │ show, book       │
├──────────┼──────────────────┤
│ PRP      │ i                │
├──────────┼──────────────────┤
│ VBP      │ am               │
├──────────┼──────────────────┤
│ VBG      │ watching         │
├──────────┼──────────────────┤
│ VB       │ show             │
├──────────┼──────────────────┤
│ DT       │ a, the           │
├──────────┼──────────────────┤
│ MD       │ will             │
╘══════════╧══════════════════╛


---
## Step 2

In [52]:
class Node:
    def __init__(self, value, parent, children):
        self.value = value
        self.parent = parent
        self.children = children

    def __str__(self):
        seq = ' (' + self.value
        for child in self.children:
            seq += str(child)
        seq += ')'

        return seq

In [53]:
def get_parents(symbol):
    parents = []

    for parent in pos_table:
        if symbol in pos_table[parent]:
            parents.append(parent)

    return list(set(parents))

In [54]:
def get_sequence(parent, symbol):
    for production in productions:
        nonterminal, sequence = production.split(' -> ')

        if nonterminal != parent:
            continue

        sequence = re.sub('\'', '', sequence).split()
        if symbol != sequence[0]:
            continue

        return sequence

In [55]:
def get_sequences(parent, symbol):
    sequences = []
    for production in productions:
        nonterminal, sequence = production.split(' -> ')

        if nonterminal != parent:
            continue

        sequence = re.sub('\'', '', sequence).split()
        if symbol != sequence[0]:
            continue

        sequences.append(sequence)

    return sequences

In [56]:
import copy

def bkt(symbol, root):
    node = Node(symbol, None, [])

    if symbol == root.value:
        return [root]

    parents = get_parents(symbol)
    nodes = []

    for parent in parents:
        parent_nodes = bkt(parent, root)
        for parent_node in parent_nodes:
            node1 = copy.deepcopy(node)

            node1.parent = parent_node
            parent_node.children.append(node1)

#             sequences = get_sequences(parent, symbol)
#             for sequence in sequences:
            sequence = get_sequence(parent, symbol)
            for nonterminal in sequence[1:]:
                node2 = Node(nonterminal, parent_node, [])
                parent_node.children.append(node2)
                nonterminals.append(node2)

            nodes.append(node1)

    return nodes

In [57]:
root = Node('S', None, [])
nonterminals = [root]

i = 0
while nonterminals:
    node = nonterminals.pop(0)
    bkt(sent[i], node)
    i += 1
    
    if i == len(sent):
        break

print(root)

 (S (NP (PRP (i))) (VP (VBP (am)) (VP (VBG (watching)) (NP (DT (a)) (NN (show))))))
